In [1]:
import os
import sys
sys.path.append('../')

from IPython.core.pylabtools import figsize

#from utils import get_intervals
from utils import get_scanpy_lfcs, get_seurat_lfcs, get_ZILN_lfcs, transform, compute_p_vals

In [2]:
import utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.stats.multitest as smm
from sklearn.metrics import confusion_matrix
import scanpy as sc

In [3]:
def log_transform(x, scale_factor=10000):
    norm_x = np.log(scale_factor * x/x.sum(0))
    return(norm_x)

# x numpy vector of counts
# transformation to apply to the non-zero elements of x.
def process_counts(x, transformation):
    cell_count = len(x)
    N_0 = np.sum(x == 0)
    N_plus = cell_count - N_0
    t_x = transformation(x[np.where(x > 0)])    
    return(t_x, N_plus, N_0)

# return p-value -- we are using Normal assumption on the means for inference. The null is that the means are the same.
def infer_lfc(m_x, s_x, m_y, s_y):
    diff = m_x - m_y 
    s_hat = np.sqrt((s_x**2) + (s_y**2))
    lb = diff - 1.96*s_hat
    ub = diff + 1.96*s_hat
    # Compute the p-value.
    z_score = diff / s_hat
    p_value = 2 * (1 - stats.norm.cdf(abs(z_score)))
    return(lb,ub,p_value)


In [11]:
# Read in the counts data
data_folder = "../simul/test/"
X = pd.read_csv(os.path.join(data_folder, "x.csv")).to_numpy()
Y = pd.read_csv(os.path.join(data_folder, "y.csv")).to_numpy()
#cell_counts_X = X.sum(axis=1)
#cell_counts_Y = Y.sum(axis=1)
n_genes = X.shape[0]
n_cells = Y.shape[1]
print("n_genes: ", n_genes)
print("n_cells: ", n_cells)
print("shape X:", X.shape)
print("shape Y:", Y.shape)



n_genes:  2
n_cells:  1222
shape X: (2, 2651)
shape Y: (2, 1222)


In [12]:
X = X.transpose()
Y = Y.transpose()

print("shape Y:", Y.shape)

shape Y: (1222, 2)


In [13]:
ziln_lfcs, p_vals = get_ZILN_lfcs(np.exp(transform(Y)) - 1, np.exp(transform(X)) - 1, eps=1e-30, return_p_vals=True)

In [14]:
ziln_lfcs

array([ 2.52652756, -0.64112454])

In [19]:
np.mean(Y[:,0] == 0)

0.9214402618657938

In [20]:
np.mean(X[:,0] == 0)

0.010562052055827989